# manually check all segmentationmap

In [ ]:
import  numpy               as     np
from    scripts.tools       import *
from    scripts.masking     import *
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
import  os
import  matplotlib.pyplot   as     plt
import  time
%matplotlib auto


obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    if obj['manual_select'] == 'selected': continue
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        seg     = find_data('SEG',hdu)[1]
        if find_data('SEG_MOD',hdu) != None: seg = find_data('SEG_MOD',hdu)[1]
        seg.data = np.where(seg.data == obj['ID'],obj['ID'],0)
        line_maps = [find_data('DSCI',hdu)[1],
                     find_data('LINE_HA',hdu)[1],
                     find_data('LINE_HB',hdu)[1]]
        seg_var = run_segmentation_editor(line_maps,seg,obj['ID'])
        seg_var.name = 'SEG_MOD'
        seg_var.writeto(f"seg/{file_name(obj,'seg')}",overwrite=True)
        hdu = save_update(seg_var,hdu)
        hdu.flush()

Using matplotlib backend: qtagg


  0%|          | 0/158 [00:00<?, ?it/s]

In [5]:
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
from    scripts.tools       import *
obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        if find_data('SEG_MOD',hdu) != None: 
            seg = find_data('SEG_MOD',hdu)[1]
        else:
            seg     = find_data('SEG',hdu)[1]
        seg.writeto(f"seg/{file_name(obj,'seg')}",overwrite=True)


100%|██████████| 158/158 [00:05<00:00, 28.34it/s]


In [2]:
from    astropy.table       import Table
from    astropy.io          import fits
from    tqdm                import tqdm
from    scripts.tools       import *
obj_lis = Table.read('obj_lis_selected.fits')
for obj in tqdm(obj_lis):
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    path_seg = f"seg/{file_name(obj,'seg')}"
    with fits.open(path,mode='update') as hdu:
        with fits.open(path_seg) as seg:
            seg[1].name = 'SEG_MOD'
            save_update(seg[1],hdu)
            hdu.flush()

100%|██████████| 158/158 [00:59<00:00,  2.66it/s]
